In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_ind, chi2_contingency
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
# import ace_tools_open as tools
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Laminitis_27_Oct/preprocessed.xlsx')
# Loop only over object (string) columns
for col in df.select_dtypes(include='number').columns:
    num_missing = df[col].isna().sum()
    if num_missing > 0:
        print(f"{col}: {num_missing} NaN values")


# Optional: Check the data types after conversion
# print(df.dtypes)


In [ ]:
df

In [ ]:
df['Class'].value_counts()


In [ ]:
# features = ['Age(years)', 'Sex', 'Respiratoryrate', 'Rectaltemperature',
#     'Gutsounds', 'Digitalpulses', 'BodyConditionScoring(outof9)',
#     'LengthLF', 'WidthLF', 'HTRF','HTRH', 'LERF','LELF','LERH', 'LELH']
# x = df[features]
# df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
x = df.drop(columns=['Class'])  # all features
y = df['Class']                 # label
# # Apply SMOTE
# smote = SMOTE(random_state=42)
# x, y = smote.fit_resample(x, y)


In [ ]:
# Check new class distribution
print("Class distribution after SMOTE:")
print(pd.Series(y).value_counts())


In [ ]:
# x = df.iloc[:,:25]
# y = df['Class']
# x.shape, y.shape

In [ ]:
# Calculate the correlation matrix
correlation_matrix = x.corr()

# Display the correlation matrix
display(correlation_matrix)

# Optional: Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Features in x')
plt.savefig('correlation_matrix.png',dpi=300)
plt.show()

In [ ]:
df.columns

In [ ]:
features = ['Age(years)', 'Sex', 'Respiratoryrate', 'Rectaltemperature',
            'Gutsounds', 'Digitalpulses', 'Bodyweight(kg)',
            'BodyConditionScoring(outof9)',
            'LengthRH',  'LengthLH', 'WidthLF','WidthRF', 'HTRF', 'HTLH',
            'LERF', 'LELF', 'LERH', 'LELH']

In [ ]:
# Instantiate and fit the Logistic Regression model
logreg = LogisticRegression()

# # Identify columns with non-numeric values
# for col in x.columns:
#     try:
#         x[col] = pd.to_numeric(x[col])
#     except ValueError:
#         print(f"Column '{col}' contains non-numeric values.")

# Convert non-numeric values to NaN and then fill with 0
# x = x.apply(pd.to_numeric, errors='coerce').fillna(0)
x = x[features]
logreg.fit(x, y)

# Access the coefficients and intercept
coefficients = logreg.coef_[0]
intercept = logreg.intercept_[0]

print("Coefficients:", coefficients)
print("Intercept:", intercept)

# Optional: Associate coefficients with feature names
feature_names = x.columns
coef_df = pd.DataFrame(zip(feature_names, coefficients), columns=['features', 'Coefficient'])
print("\nCoefficients with Feature Names:")
print(coef_df)

In [ ]:
def convert_to_beta_dict(feature_names, coefficients, round_digits=6):
    """
    Converts feature names and coefficients into a dictionary.

    Args:
        feature_names (list): List of feature/column names.
        coefficients (list or array): Corresponding list/array of coefficients.
        round_digits (int): Number of decimal digits to round the coefficients.

    Returns:
        dict: Dictionary mapping feature names to rounded coefficient values.
    """
    return dict(zip(feature_names, [round(float(coef), round_digits) for coef in coefficients]))

beta_values = convert_to_beta_dict(feature_names, coefficients)

print(beta_values)

In [ ]:
def find_column_ranges(df):
    """
    Finds the range (min and max) for all columns in a DataFrame, excluding 'lameness_risk_binary'.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the min and max values for each column.
    """
    # Select columns excluding 'lameness_risk_binary'
    numeric_columns = df.columns

    # Calculate the min and max for each column
    min_values = df[numeric_columns].min()
    max_values = df[numeric_columns].max()

    # Create a DataFrame to store the results
    range_df = pd.DataFrame({'Min': min_values, 'Max': max_values})

    return range_df

# Assuming your DataFrame is named 'df'
column_ranges = find_column_ranges(x)
print(column_ranges)

In [ ]:
features = ['Age(years)', 'Sex', 'Respiratoryrate', 'Rectaltemperature',
            'Gutsounds', 'Digitalpulses', 'Bodyweight(kg)',
            'BodyConditionScoring(outof9)',
            'LengthRH',  'LengthLH', 'WidthLF','WidthRF', 'HTRF', 'HTLH',
            'LERF', 'LELF', 'LERH', 'LELH']

In [ ]:
import pandas as pd

# Define reference categories and Wj values for all features
reference_data_all = [
    # Age(years)
    ["Age(years)", "0-4", 2],
    ["Age(years)", "5-9", 7],
    ["Age(years)", "10-14", 12],
    ["Age(years)", "15-20", 17],
    ["Age(years)", "21-25", 23],

    # Sex (categorical: 0–3)
    *[["Sex", str(sex), sex] for sex in range(0, 4)],

    # Respiratory Rate
    ["Respiratoryrate", "8-16", 12],
    ["Respiratoryrate", "17-30", 24],
    ["Respiratoryrate", "31-50", 40],
    ["Respiratoryrate", "51-76", 63],

    # Rectal Temperature
    ["Rectaltemperature", "35.8-36.9", 36.4],
    ["Rectaltemperature", "37.0-38.0", 37.5],
    ["Rectaltemperature", "38.1-38.5", 38.3],

    # Gutsounds (binary)
    ["Gutsounds", "0", 0],
    ["Gutsounds", "1", 1],

    # Digitalpulses (binary)
    ["Digitalpulses", "0", 0],
    ["Digitalpulses", "1", 1],

    # Bodyweight(kg)
    ["Bodyweight(kg)", "245-300", 272],
    ["Bodyweight(kg)", "301-400", 350],
    ["Bodyweight(kg)", "401-500", 450],
    ["Bodyweight(kg)", "501-567", 534],

    # Body Condition Scoring(outof9)
    ["BodyConditionScoring(outof9)", "1-3", 2],
    ["BodyConditionScoring(outof9)", "4-6", 5],
    ["BodyConditionScoring(outof9)", "7-9", 8],

    # Length features
    *[
        [feature, "0-5", 2.5] for feature in ['LengthRH',  'LengthLH']
    ] + [
        [feature, "5.1-10", 7.5] for feature in ['LengthRH',  'LengthLH']
    ] + [
        [feature, "10.1-14.5", 12.3] for feature in ['LengthRH',  'LengthLH']
    ],

    # Width features
    *[
        [feature, "0-5", 2.5] for feature in ['WidthLF','WidthRF']
    ] + [
        [feature, "5.1-10", 7.5] for feature in ['WidthLF','WidthRF']
    ] + [
        [feature, "10.1-14.0", 12.0] for feature in ['WidthLF','WidthRF']
    ],

    # Heat in feet (ordinal 0–3)
    # Heat in feet (ordinal, based on actual data)

        ["HTRF", "0", 0],
        ["HTRF", "1", 1],
        ["HTRF", "2", 2],

        ["HTLF", "0", 0],
        ["HTLF", "1", 1],
        ["HTLF", "2", 2],
        ["HTLF", "3", 3],

        ["HTRH", "0", 0],
        ["HTRH", "1", 1],

        ["HTLH", "0", 0],
        ["HTLH", "1", 1],

    # Lesion scores (ordinal 0–4)

        ["LERF", "0", 0],
        ["LERF", "3", 3],
        ["LERF", "4", 4],

        ["LELF", "0", 0],
        ["LELF", "3", 3],
        ["LELF", "4", 4],

        ["LERH", "0", 0],
        ["LERH", "3", 3],
        ["LERH", "4", 4],

        ["LELH", "0", 0],
        ["LELH", "3", 3],
        ["LELH", "4", 4],
]

# Create the reference table as a DataFrame
reference_table = pd.DataFrame(reference_data_all, columns=["Risk factor", "Categories", "Reference value (Wj)"])

# Example: print the first few rows
reference_table


In [ ]:
import pandas as pd


def calculate_beta_values(reference_table, beta_values):
    """
    Calculates the beta * (value - base) for each row in the reference table
    and adds it as a new column, also prints the beta value in the table.

    Args:
        reference_table (pd.DataFrame): DataFrame containing risk factors,
                                         categories, and reference values.
        beta_values (dict): Dictionary containing beta values for each risk factor.

    Returns:
        pd.DataFrame: DataFrame with the new 'Beta', and 'Beta * (Value - Base)' columns,
                      or None if the DataFrame is empty.
    """

    if reference_table.empty:
        print("DataFrame is empty. Cannot calculate beta values.")
        return None

    # Create a copy to avoid modifying the original DataFrame
    reference_table = reference_table.copy()

    # Create a dictionary to store the base values for each risk factor
    base_values = {}
    for risk_factor in reference_table['Risk factor'].unique():
        base_values[risk_factor] = reference_table[reference_table['Risk factor'] == risk_factor]['Reference value (Wj)'].iloc[0]

    # Define a function to calculate the beta value for a given row
    def calculate_beta(row):
        risk_factor = row['Risk factor']
        reference_value = row['Reference value (Wj)']
        beta = beta_values.get(risk_factor)  # Use .get() to handle missing betas

        if beta is not None:
            return beta, beta * (reference_value - base_values[risk_factor])
        else:
            return None, None  # Or NaN, or a specific error value

    # Apply the function to each row to create the new columns
    reference_table[['Beta', 'Beta * (Value - Base)']] = reference_table.apply(calculate_beta, axis=1, result_type='expand')

    return reference_table
# def calculate_beta_values(reference_table, beta_values):
#     """
#     Calculates the beta * (value - base) for each row in the reference table
#     and adds it as a new column, also prints the beta value in the table.

#     Args:
#         reference_table (pd.DataFrame): DataFrame containing risk factors,
#                                          categories, and reference values.
#         beta_values (dict): Dictionary containing beta values for each risk factor.

#     Returns:
#         pd.DataFrame: DataFrame with the new 'Beta', and 'Beta * (Value - Base)' columns,
#                       or None if the DataFrame is empty.
#     """

#     if reference_table.empty:
#         print("DataFrame is empty. Cannot calculate beta values.")
#         return None

#     # Create a copy to avoid modifying the original DataFrame
#     reference_table = reference_table.copy()

#     # Create a dictionary to store the base values for each risk factor
#     base_values = {}
#     for risk_factor in reference_table['Risk factor'].unique():
#         base_values[risk_factor] = reference_table[reference_table['Risk factor'] == risk_factor]['Reference value (Wj)'].iloc[0]

#     # Define a function to calculate the beta value for a given row
#     def calculate_beta(row):
#         risk_factor = row['Risk factor']
#         reference_value = row['Reference value (Wj)']
#         beta = beta_values.get(risk_factor)  # Use .get() to handle missing betas

#         if beta is not None:
#             return beta, beta * (reference_value - base_values[risk_factor])
#         else:
#             return None, None  # Or NaN, or a specific error value

#     # Apply the function to each row to create the new columns
#     reference_table[['Beta', 'Beta * (Value - Base)']] = reference_table.apply(calculate_beta, axis=1, result_type='expand')

#     return reference_table

reference_table_with_beta = calculate_beta_values(reference_table, beta_values)
reference_table_with_beta

In [ ]:
# # STEP 2: Your feature names in order
# feature_names = features
# # STEP 3: Reference table you already created
# # Make sure `reference_table` has columns: ['Risk factor', 'Categories', 'Reference value (Wj)']
# reference_table["β_i"] = reference_table["Risk factor"].map(dict(zip(feature_names, coefficients)))

# # STEP 4: Assign W_REF for each feature (first category assumed as reference)
# w_ref_map = reference_table.groupby("Risk factor")["Reference value (Wj)"].first().to_dict()
# reference_table["W_REF"] = reference_table["Risk factor"].map(w_ref_map)

# # STEP 5: Compute β_i * (W_ij - W_REF)
# reference_table["β(Wij - W_REF)"] = reference_table["β_i"] * (reference_table["Reference value (Wj)"] - reference_table["W_REF"])

# # STEP 6: Calculate B as the minimum non-zero absolute value of the difference
# non_zero_diffs = reference_table["β(Wij - W_REF)"]
# non_zero_diffs = non_zero_diffs[(non_zero_diffs != 0) & (~non_zero_diffs.isna())].abs()
# B = non_zero_diffs.min()

B = 5 * reference_table_with_beta['Beta'][1]

# Output B
print("Calculated B (Framingham method):", B)


In [ ]:
def calculate_points_from_precalculated(beta_times_diff, B):
  """Calculates the points using the precalculated Beta * (Value - Base) and rounds to the nearest integer."""
  if pd.isna(beta_times_diff):
    return 0 # Or np.nan if you prefer to keep them as missing

  points = beta_times_diff / B
  return int(np.round(points))

# Apply the calculate_points_from_precalculated function to each row:
reference_table_with_beta['Points'] = reference_table_with_beta.apply(lambda row: calculate_points_from_precalculated(row['Beta * (Value - Base)'], B), axis=1)

# Print the DataFrame with the calculated points:
reference_table_with_beta

In [ ]:
import numpy as np
import pandas as pd
# Intercept: 0.0112847841120903
def calculate_risk(point_total, intercept=intercept, constant=B):
  """
  Calculates the risk associated with a given point total.

  Args:
    point_total: The point total.
    intercept: The intercept value (β₀).
    constant: The constant B.

  Returns:
    The calculated risk (p̂).
  """
  # Calculate ΣβX
  sum_beta_x = intercept + (constant * point_total)

  # Calculate risk (p̂)
  p_hat = 1 / (1 + np.exp(-sum_beta_x))
  return p_hat

# Define the range of point totals
point_totals = range(min(reference_table_with_beta['Points']),max(reference_table_with_beta['Points'])+2)  # Generates numbers from -7 to 9

# Calculate the risk for each point total and store the results
results = []
for point_total in point_totals:
  risk = calculate_risk(point_total)
  results.append({'Point Total': point_total, 'Risk': risk})

# Create a Pandas DataFrame to display the results
risk_estimate = pd.DataFrame(results)

# Print the DataFrame
print(risk_estimate)

In [ ]:
# Merge the two DataFrames to add the 'Risk' column
reference_table_with_risk = pd.merge(
    reference_table_with_beta,
    risk_estimate,
    left_on='Points',
    right_on='Point Total',
    how='left'  # Use a left merge to keep all rows from reference_table_with_beta
)

# Drop the redundant 'Point Total' column
reference_table_with_risk = reference_table_with_risk.drop(columns=['Point Total'])

# Round the 'Risk' column to two decimal places
reference_table_with_risk['Risk'] = reference_table_with_risk['Risk'].round(2)

# Display the updated DataFrame
reference_table_with_risk

## Table

In [ ]:
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE
import pandas as pd

# Assuming your features and label are:
x = df.drop(columns=['Class'])  # all features
X = x.apply(pd.to_numeric, errors='coerce').fillna(0)
y = df['Class']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

def calculate_custom_risk_score(df):
    """
    Calculate custom risk score based on your available features
    This adapts the PRISQ concept to your specific features
    """
    scores = []

    for _, row in df.iterrows():
        score = 0

        # Age scoring (adapted from PRISQ)
        age = row['Age(years)']
        if 10 <= age <= 15:  # Adjusted for animal ages
            score += 8
        elif age > 15:
            score += 15

        # Sex scoring
        sex = row['Sex']
        # Assuming 1 = male, 0 = female or similar encoding
        if sex == 1:
            score += 3

        # Body weight scoring
        weight = row['Bodyweight(kg)']
        if weight > 500:  # Adjust threshold based on your data distribution
            score += 4
        elif weight > 400:
            score += 2

        # Body condition scoring
        bcs = row['BodyConditionScoring(outof9)']
        if bcs >= 7:  # Overweight/obese
            score += 5
        elif bcs >= 5:  # Moderate condition
            score += 2

        # Temperature scoring (abnormal temperatures)
        temp = row['Rectaltemperature']
        if temp < 37 or temp > 38.5:  # Abnormal temperature range
            score += 3

        # Respiratory rate scoring
        resp_rate = row['Respiratoryrate']
        if resp_rate > 20:  # Elevated respiratory rate
            score += 2

        # Digital pulses scoring (assuming abnormal = higher risk)
        # This might indicate circulatory issues
        digital_pulses = row['Digitalpulses']
        if digital_pulses > 2:  # Adjust based on your scale
            score += 3

        # Add limb measurements if they indicate asymmetry or abnormalities
        # Calculate asymmetry between left and right limbs
        length_asymmetry = abs(row['LengthRH'] - row['LengthLH'])
        width_asymmetry = abs(row['WidthLF'] - row['WidthRF'])

        if length_asymmetry > 1.0 or width_asymmetry > 0.5:  # Adjust thresholds
            score += 2

        scores.append(score)

    return scores

# Alternative: Use machine learning to create risk scores
def calculate_ml_risk_score(X_train, X_test, y_train):
    """
    Use Random Forest to create risk scores based on feature importance
    """
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Get predicted probabilities for positive class
    train_proba = rf.predict_proba(X_train)[:, 1]
    test_proba = rf.predict_proba(X_test)[:, 1]

    # Convert probabilities to scores (0-100 scale)
    train_scores = (train_proba * 100).astype(int)
    test_scores = (test_proba * 100).astype(int)

    return train_scores, test_scores, rf

In [ ]:
def create_custom_table_3(X_train, X_test, y_train, y_test, method='custom'):
    """
    Create Table 3 for your data using custom risk scoring
    """

    if method == 'custom':
        # Use custom scoring system
        train_scores = calculate_custom_risk_score(X_train)
        test_scores = calculate_custom_risk_score(X_test)
    else:
        # Use ML-based scoring
        train_scores, test_scores, model = calculate_ml_risk_score(X_train, X_test, y_train)

    # Define risk categories based on your score distribution
    def categorize_risk(score):
        # Adjust these thresholds based on your score distribution
        if score <= np.percentile(train_scores, 33):  # Lower third
            return 'Low Risk'
        elif score <= np.percentile(train_scores, 66):  # Middle third
            return 'Moderate Risk'
        else:  # Upper third
            return 'High Risk'

    # Categorize risks
    train_risk = [categorize_risk(score) for score in train_scores]
    test_risk = [categorize_risk(score) for score in test_scores]

    # Create results DataFrames
    train_df = pd.DataFrame({
        'score': train_scores,
        'risk_category': train_risk,
        'actual': y_train
    })

    test_df = pd.DataFrame({
        'score': test_scores,
        'risk_category': test_risk,
        'actual': y_test
    })

    # Calculate Table 3 statistics
    def calculate_category_stats(df, dataset_name):
        results = []
        for category in ['Low Risk', 'Moderate Risk', 'High Risk']:
            category_data = df[df['risk_category'] == category]
            total = len(category_data)
            if total > 0:
                controls = len(category_data[category_data['actual'] == 0])
                cases = len(category_data[category_data['actual'] == 1])
                prevalence = (cases / total) * 100
            else:
                controls = cases = prevalence = 0

            results.append({
                'Dataset': dataset_name,
                'Risk Level': category,
                'Controls': controls,
                'Cases': cases,
                'Total': total,
                'Prevalence (%)': round(prevalence, 2)
            })
        return results

    # Generate table data
    table_data = (calculate_category_stats(train_df, 'Training') +
                  calculate_category_stats(test_df, 'Testing'))

    table_3 = pd.DataFrame(table_data)

    return table_3, train_df, test_df

# Usage
table_3, train_df, test_df = create_custom_table_3(X_train, X_test, y_train, y_test, method='custom')
print("Your Custom Table 3:")
print(table_3)

In [ ]:
def plot_custom_table_3(table_3, title_suffix="Your Dataset"):
    """
    Plot your custom Table 3 data
    """
    # Separate training and testing data
    train_data = table_3[table_3['Dataset'] == 'Training']
    test_data = table_3[table_3['Dataset'] == 'Testing']

    # Create subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # Plot 1: Prevalence bar chart (Training)
    categories = train_data['Risk Level'].tolist()
    train_prevalence = train_data['Prevalence (%)'].tolist()

    bars1 = ax1.bar(categories, train_prevalence,
                   color=['green', 'orange', 'red'], alpha=0.7)
    ax1.set_title(f'Training Dataset - Prevalence by Risk Category\n{title_suffix}',
                  fontsize=12, fontweight='bold')
    ax1.set_ylabel('Prevalence (%)')
    max_prevalence = max(train_prevalence) if train_prevalence else 100
    ax1.set_ylim(0, max_prevalence * 1.2)

    # Add value labels
    for bar, value in zip(bars1, train_prevalence):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value}%', ha='center', va='bottom', fontweight='bold')

    # Plot 2: Prevalence bar chart (Testing)
    test_prevalence = test_data['Prevalence (%)'].tolist()

    bars2 = ax2.bar(categories, test_prevalence,
                   color=['green', 'orange', 'red'], alpha=0.7)
    ax2.set_title(f'Testing Dataset - Prevalence by Risk Category\n{title_suffix}',
                  fontsize=12, fontweight='bold')
    ax2.set_ylabel('Prevalence (%)')
    max_prevalence_test = max(test_prevalence) if test_prevalence else 100
    ax2.set_ylim(0, max_prevalence_test * 1.2)

    for bar, value in zip(bars2, test_prevalence):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value}%', ha='center', va='bottom', fontweight='bold')

    # Plot 3: Stacked bar chart (Training)
    train_controls = train_data['Controls'].tolist()
    train_cases = train_data['Cases'].tolist()
    train_totals = [c + cs for c, cs in zip(train_controls, train_cases)]

    ax3.bar(categories, train_controls, label='Controls', color='lightblue', alpha=0.8)
    ax3.bar(categories, train_cases, bottom=train_controls, label='Cases', color='coral', alpha=0.8)
    ax3.set_title('Training Dataset - Distribution', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Number of Subjects')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # Add prevalence percentages - FIXED THIS PART
    for i, (control, case, prev, total) in enumerate(zip(train_controls, train_cases, train_prevalence, train_totals)):
        ax3.text(i, total + max(train_totals)*0.05, f'{prev}%', ha='center', va='bottom',
                fontweight='bold', fontsize=10)

    # Plot 4: Stacked bar chart (Testing)
    test_controls = test_data['Controls'].tolist()
    test_cases = test_data['Cases'].tolist()
    test_totals = [c + cs for c, cs in zip(test_controls, test_cases)]

    ax4.bar(categories, test_controls, label='Controls', color='lightblue', alpha=0.8)
    ax4.bar(categories, test_cases, bottom=test_controls, label='Cases', color='coral', alpha=0.8)
    ax4.set_title('Testing Dataset - Distribution', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Number of Subjects')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # Add prevalence percentages - FIXED THIS PART
    for i, (control, case, prev, total) in enumerate(zip(test_controls, test_cases, test_prevalence, test_totals)):
        ax4.text(i, total + max(test_totals)*0.05, f'{prev}%', ha='center', va='bottom',
                fontweight='bold', fontsize=10)

    plt.tight_layout()
    plt.show()

    return fig

# Generate the plots
fig = plot_custom_table_3(table_3, "Your Clinical Features")

In [ ]:
def analyze_risk_stratification(table_3):
    """
    Analyze how well your risk stratification works
    """
    print("=== RISK STRATIFICATION ANALYSIS ===")

    for dataset in ['Training', 'Testing']:
        data = table_3[table_3['Dataset'] == dataset]
        low_risk_prev = data[data['Risk Level'] == 'Low Risk']['Prevalence (%)'].values[0]
        high_risk_prev = data[data['Risk Level'] == 'High Risk']['Prevalence (%)'].values[0]

        risk_ratio = high_risk_prev / low_risk_prev if low_risk_prev > 0 else float('inf')

        print(f"\n{dataset} Dataset:")
        print(f"Low Risk Prevalence: {low_risk_prev}%")
        print(f"High Risk Prevalence: {high_risk_prev}%")
        print(f"Risk Ratio (High/Low): {risk_ratio:.2f}")

        if risk_ratio > 2:
            print("✓ Good risk stratification")
        elif risk_ratio > 1.5:
            print("○ Moderate risk stratification")
        else:
            print("✗ Poor risk stratification")

# Run analysis
analyze_risk_stratification(table_3)

## Second method

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def calculate_custom_risk_score(df):
    """
    CORRECT VERSION: Calculate risk score using your actual features
    """
    scores = []

    for _, row in df.iterrows():
        score = 0

        # 1. AGE SCORING (High importance)
        age = row['Age(years)']
        if age >= 15:    # Older animals higher risk
            score += 20
        elif age >= 10:
            score += 12
        elif age >= 5:
            score += 6

        # 2. BODY CONDITION SCORING (High importance)
        bcs = row['BodyConditionScoring(outof9)']
        if bcs >= 7:     # Overweight/obese
            score += 8
        elif bcs >= 5:   # Moderate
            score += 4
        # Low BCS (<5) gets 0 points

        # 3. SEX SCORING
        sex = row['Sex']
        if sex == 1:     # Male (adjust if your encoding is different)
            score += 3

        # 4. VITAL SIGNS SCORING
        # Heart rate (adjust thresholds based on normal ranges for your species)
        hr = row['HeartRate']
        if hr > 50 or hr < 30:  # Abnormal heart rate
            score += 3

        # Respiratory rate
        resp = row['Respiratoryrate']
        if resp > 25 or resp < 10:  # Abnormal respiratory rate
            score += 3

        # Temperature
        temp = row['Rectaltemperature']
        if temp < 37.0 or temp > 38.5:  # Abnormal temperature
            score += 3

        # 5. DIGITAL PULSES (circulatory assessment)
        pulses = row['Digitalpulses']
        if pulses > 2:  # Abnormal pulses (adjust threshold)
            score += 4

        # 6. BODY WEIGHT
        weight = row['Bodyweight(kg)']
        if weight > 450:  # Heavy weight (adjust threshold)
            score += 3

        scores.append(score)

    return scores

def create_final_risk_categories(X_train, X_test, y_train, y_test):
    """
    FINAL CORRECT VERSION: Create risk categories with validation
    """
    # Calculate scores
    train_scores = calculate_custom_risk_score(X_train)
    test_scores = calculate_custom_risk_score(X_test)

    print("=== SCORE DISTRIBUTION ===")
    print(f"Training - Min: {min(train_scores)}, Max: {max(train_scores)}, Mean: {np.mean(train_scores):.2f}")
    print(f"Testing  - Min: {min(test_scores)}, Max: {max(test_scores)}, Mean: {np.mean(test_scores):.2f}")

    # Use percentile-based categorization (MOST RELIABLE APPROACH)
    p33 = np.percentile(train_scores, 33)
    p66 = np.percentile(train_scores, 66)

    print(f"\n=== AUTOMATIC RISK THRESHOLDS ===")
    print(f"Low Risk: score ≤ {p33:.1f}")
    print(f"Moderate Risk: {p33:.1f} < score ≤ {p66:.1f}")
    print(f"High Risk: score > {p66:.1f}")

    def categorize_risk(scores):
        categories = []
        for score in scores:
            if score <= p33:
                categories.append('Low Risk')
            elif score <= p66:
                categories.append('Moderate Risk')
            else:
                categories.append('High Risk')
        return categories

    # Apply categorization
    train_risk = categorize_risk(train_scores)
    test_risk = categorize_risk(test_scores)

    # Create results DataFrames
    train_df = pd.DataFrame({
        'score': train_scores,
        'risk_category': train_risk,
        'actual': y_train.values
    })

    test_df = pd.DataFrame({
        'score': test_scores,
        'risk_category': test_risk,
        'actual': y_test.values
    })

    return train_df, test_df, train_scores, test_scores

# RUN THE CORRECT IMPLEMENTATION
print("CREATING RISK CATEGORIES...")
train_df, test_df, train_scores, test_scores = create_final_risk_categories(X_train, X_test, y_train, y_test)

print("\n✅ RISK CATEGORIES CREATED SUCCESSFULLY!")

def create_verified_table_3(train_df, test_df):
    """
    CREATE VERIFIED TABLE 3 for your data
    """
    def calculate_stats(df, dataset_name):
        results = []
        for category in ['Low Risk', 'Moderate Risk', 'High Risk']:
            cat_data = df[df['risk_category'] == category]
            total = len(cat_data)
            controls = len(cat_data[cat_data['actual'] == 0])
            cases = len(cat_data[cat_data['actual'] == 1])
            prevalence = (cases / total * 100) if total > 0 else 0

            results.append({
                'Dataset': dataset_name,
                'Risk Level': category,
                'Controls': controls,
                'Cases': cases,
                'Total': total,
                'Prevalence (%)': round(prevalence, 2)
            })
        return results

    # Generate table data
    table_data = calculate_stats(train_df, 'Training') + calculate_stats(test_df, 'Testing')
    table_3 = pd.DataFrame(table_data)

    return table_3

# Create your Table 3
table_3 = create_verified_table_3(train_df, test_df)
print("\nYOUR TABLE 3:")
print(table_3)


def plot_final_results(table_3):
    """
    ERROR-FREE plotting function
    """
    # Separate data
    train_data = table_3[table_3['Dataset'] == 'Training']
    test_data = table_3[table_3['Dataset'] == 'Testing']

    categories = train_data['Risk Level'].tolist()
    train_prev = train_data['Prevalence (%)'].tolist()
    test_prev = test_data['Prevalence (%)'].tolist()

    # Create plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Training data
    bars1 = ax1.bar(categories, train_prev, color=['green', 'orange', 'red'], alpha=0.7)
    ax1.set_title('Training Dataset\nPrediabetes Prevalence by Risk Category', fontweight='bold')
    ax1.set_ylabel('Prevalence (%)')
    ax1.set_ylim(0, max(train_prev + test_prev) * 1.2)

    for bar, value in zip(bars1, train_prev):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value}%', ha='center', va='bottom', fontweight='bold')

    # Testing data
    bars2 = ax2.bar(categories, test_prev, color=['green', 'orange', 'red'], alpha=0.7)
    ax2.set_title('Testing Dataset\nPrediabetes Prevalence by Risk Category', fontweight='bold')
    ax2.set_ylabel('Prevalence (%)')
    ax2.set_ylim(0, max(train_prev + test_prev) * 1.2)

    for bar, value in zip(bars2, test_prev):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value}%', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.show()

    return fig

# Plot your results
print("\nGENERATING PLOTS...")
fig = plot_final_results(table_3)

def validate_results(train_df, test_df, table_3):
    """
    VALIDATE that everything is correct
    """
    print("\n" + "="*60)
    print("VALIDATION RESULTS")
    print("="*60)

    # Check category distributions
    print("\nTRAINING DATA DISTRIBUTION:")
    train_counts = train_df['risk_category'].value_counts()
    for category in ['Low Risk', 'Moderate Risk', 'High Risk']:
        count = train_counts.get(category, 0)
        print(f"  {category}: {count} subjects ({count/len(train_df)*100:.1f}%)")

    print("\nTESTING DATA DISTRIBUTION:")
    test_counts = test_df['risk_category'].value_counts()
    for category in ['Low Risk', 'Moderate Risk', 'High Risk']:
        count = test_counts.get(category, 0)
        print(f"  {category}: {count} subjects ({count/len(test_df)*100:.1f}%)")

    # Check risk stratification
    print("\nRISK STRATIFICATION PERFORMANCE:")
    for dataset in ['Training', 'Testing']:
        data = table_3[table_3['Dataset'] == dataset]
        low_risk_prev = data[data['Risk Level'] == 'Low Risk']['Prevalence (%)'].values[0]
        high_risk_prev = data[data['Risk Level'] == 'High Risk']['Prevalence (%)'].values[0]

        risk_ratio = high_risk_prev / low_risk_prev if low_risk_prev > 0 else float('inf')

        print(f"  {dataset}: Low Risk = {low_risk_prev}%, High Risk = {high_risk_prev}%")
        print(f"  Risk Ratio (High/Low): {risk_ratio:.2f}x")

    print("="*60)

# Run validation
validate_results(train_df, test_df, table_3)